In [1]:
!pip install langchain openai tiktoken rapidocr-onnxruntime

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.3 MB 11.1 MB/s eta 0:00:01
   ------------------ --------------------- 5.8/12.3 MB 14.4 MB/s eta 0:00:01
   ---------------------------- ----------- 8.7/12.3 MB 13.9 MB/s eta 0:00:01
   ------------------------------------ --- 11.3/12.3 MB 13.7 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 12.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   --- ------------------------------------ 3.7/39.0 MB 18.0 MB/s eta 0:00:02
   ------ --------------------------------- 6.3/39.0 MB 15.4 MB/s eta 0:00:03
   -------- ------------------------------- 7.9/39.0 MB 14.8 MB/s eta 0:00:03
   ------------ --------------------------- 11.8/39.0 MB 14.1 MB/s eta 0:00:02
   --------------- ------------------------ 14.7/39.0 MB 13.9 MB/s eta 0:00:02
   ----------------- ---------------------- 17.3/39.0 MB 13.7 MB/s eta 0:00:

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

In [5]:
with open("train_data.txt","r", encoding="utf8") as f:
    data = f.read()

In [6]:
loder=TextLoader('train_data.txt' , encoding="utf8")

In [7]:
document=loder.load()

In [8]:
print(document[0].page_content)

How do you manage information across modalities?
Another crucial aspect is representing information across different modalities. For instance, if you are working with a document, you must make sure that the semantic representation of a chart aligns with the semantic representation of the text discussing the same chart.

Approaches for multimodal retrieval
With the key challenge understood, here are the specifics of building RAG pipelines to tackle these challenges.

There are several main approaches to building multi-modal RAG pipelines:

Embed all modalities into the same vector space
Ground all modalities into one primary modality
Have separate stores for different modalities
To keep this discussion concise, we only discuss images and text input.

Embed all modalities into the same vector space
In the case of images and text, you can use a model like CLIP to encode both text and images in the same vector space. This makes it so that you can largely use the same text-only RAG infrastr

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)

In [11]:
text_chunks=text_splitter.split_documents(document)

In [12]:
text_chunks

[Document(metadata={'source': 'train_data.txt'}, page_content='How do you manage information across modalities?'),
 Document(metadata={'source': 'train_data.txt'}, page_content='Another crucial aspect is representing information across different modalities. For instance, if you are working with a document, you must make sure that the semantic representation of a chart aligns'),
 Document(metadata={'source': 'train_data.txt'}, page_content='of a chart aligns with the semantic representation of the text discussing the same chart.'),
 Document(metadata={'source': 'train_data.txt'}, page_content='Approaches for multimodal retrieval\nWith the key challenge understood, here are the specifics of building RAG pipelines to tackle these challenges.'),
 Document(metadata={'source': 'train_data.txt'}, page_content='There are several main approaches to building multi-modal RAG pipelines:'),
 Document(metadata={'source': 'train_data.txt'}, page_content='Embed all modalities into the same vector spac

In [13]:
print(text_chunks[2].page_content)

of a chart aligns with the semantic representation of the text discussing the same chart.


In [14]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [16]:
!pip install langchain-openai

In [17]:
from langchain_openai import OpenAIEmbeddings

In [18]:
embeddings=OpenAIEmbeddings()

In [19]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---- ----------------------------------- 2.1/18.2 MB 14.0 MB/s eta 0:00:02
   ------------ --------------------------- 5.8/18.2 MB 16.1 MB/s eta 0:00:01
   ------------------ --------------------- 8.4/18.2 MB 14.9 MB/s eta 0:00:01
   ------------------------ --------------- 11.3/18.2 MB 14.4 MB/s eta 0:00:01
   ------------------------------ --------- 13.9/18.2 MB 14.1 MB/s eta 0:00:01
   ---------------------------------- ----- 15.7/18.2 MB 13.0 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 13.1 MB/s eta 0:00:00


In [20]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [21]:
retriever=vectorstore.as_retriever()

In [22]:
from langchain.prompts import ChatPromptTemplate

In [23]:
from langchain import hub

In [24]:
prompt = hub.pull("jclemens24/rag-prompt")

In [28]:
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [26]:
output_parser=StrOutputParser()

In [29]:
llm_model=ChatOpenAI(model_name="gpt-3.5-turbo")

In [30]:
rag_chain=(
    {"context": retriever, "question":RunnablePassthrough()}
    | prompt
    |llm_model
    |output_parser
)

In [31]:
rag_chain.invoke("Embed text in PDFs")

'To embed text in PDFs, you need to first separate images and text, classify images using an MLLM based on image types, and then embed the text into the PDFs. This process allows for capturing all intricacies like text in images and complex tables.'